In [6]:
!pip list

Package                            Version
---------------------------------- -------------------
absl-py                            1.4.0
accelerate                         1.5.2
aiohappyeyeballs                   2.6.1
aiohttp                            3.11.14
aiosignal                          1.3.2
alabaster                          1.0.0
albucore                           0.0.23
albumentations                     2.0.5
ale-py                             0.10.2
altair                             5.5.0
annotated-types                    0.7.0
anyio                              4.9.0
argon2-cffi                        23.1.0
argon2-cffi-bindings               21.2.0
array_record                       0.7.1
arviz                              0.21.0
astroid                            3.3.9
astropy                            7.0.1
astropy-iers-data                  0.2025.3.24.0.35.32
astunparse                         1.6.3
atpublic                           5.1
attrs                 

In [9]:
import requests
import re
import json


def request_dandan(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
    except requests.RequestException as e:
        print(e)
        return None


def parse_result(html):
    pattern = re.compile(
        '<li.*?list_num.*?(\d+)\.</div>.*?<img src="(.*?)".*?class="name".*?title="(.*?)">.*?class="star">.*?class="tuijian">(.*?)</span>.*?class="publisher_info">.*?target="_blank">(.*?)</a>.*?class="biaosheng">.*?<span>(.*?)</span></div>.*?<p><span class="price_n">(.*?)</span>.*?</li>', re.S)
    items = re.findall(pattern, html)

    for item in items:
        yield {
            'range': item[0],
            'image': item[1],
            'title': item[2],
            'recommend': item[3],
            'author': item[4],
            'times': item[5],
            'price': item[6]
        }


def write_item_to_file(item):
    print('开始写入数据 ====> ' + str(item))
    with open('book.txt', 'a', encoding='UTF-8') as f:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')


def main(page):
    url = 'http://bang.dangdang.com/books/fivestars/01.00.00.00.00.00-recent30-0-0-1-' + str(page)
    html = request_dandan(url)
    items = parse_result(html)  # 解析过滤我们想要的信息
    for item in items:
        write_item_to_file(item)


if __name__ == "__main__":
    for i in range(1, 26):
        main(i)

开始写入数据 ====> {'range': '1', 'image': 'http://img3m1.ddimg.cn/81/6/29849571-1_l_1742286527.jpg', 'title': '你可以有情绪，但别往心里去【破解情绪内耗的疗愈指南】赠情绪记录卡', 'recommend': '100%推荐', 'author': '班曼曼', 'times': '999次', 'price': '&yen;21.00'}
开始写入数据 ====> {'range': '2', 'image': 'http://img3m7.ddimg.cn/78/17/29842737-1_l_1737447307.jpg', 'title': '地下室手记 “地下室人”的困境是当下每个边缘人的映射', 'recommend': '100%推荐', 'author': '陀思妥耶夫斯基', 'times': '995次', 'price': '&yen;9.90'}
开始写入数据 ====> {'range': '3', 'image': 'http://img3m2.ddimg.cn/81/29/29819772-1_l_1738728197.jpg', 'title': '一本关于你的书 我写了一本关于你的书 全彩精装', 'recommend': '100%推荐', 'author': '笑羽', 'times': '3372次', 'price': '&yen;23.90'}
开始写入数据 ====> {'range': '4', 'image': 'http://img3m9.ddimg.cn/88/4/29839579-1_l_1736922819.jpg', 'title': '幸福者退让原则【罗翔强烈呼吁，并详细解读“幸福者退让原则”】', 'recommend': '100%推荐', 'author': '王辉', 'times': '2198次', 'price': '&yen;21.00'}
开始写入数据 ====> {'range': '5', 'image': 'http://img3m5.ddimg.cn/34/7/29845465-1_l_1739176206.jpg', 'title': '人类迁徙（BBC驻外记者先锋之作，另类视角的

In [15]:
import requests
from bs4 import BeautifulSoup
import xlwt


def request_douban(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                      'Chrome/88.0.4324.146 Safari/537.36',
    }

    try:
        response = requests.get(url=url, headers=headers)
        if response.status_code == 200:
            return response.text
    except requests.RequestException:
        return None


book = xlwt.Workbook(encoding='utf-8', style_compression=0)

sheet = book.add_sheet('豆瓣电影Top250', cell_overwrite_ok=True)
sheet.write(0, 0, '名称')
sheet.write(0, 1, '图片')
sheet.write(0, 2, '排名')
sheet.write(0, 3, '评分')
sheet.write(0, 4, '作者')
sheet.write(0, 5, '简介')

n = 1


def save_to_excel(soup):
    list = soup.find(class_='grid_view').find_all('li')

    for item in list:
        item_name = item.find(class_='title').string
        item_img = item.find('a').find('img').get('src')
        # item_index = item.find(class_='').string
        item_score = item.find(class_='rating_num').string
        item_author = item.find('p').text
        if item.find(class_='inq') is not None:
            item_intr = item.find(class_='inq').string
        else:
            item_intr = 'NOT AVAILABLE'

        # print('爬取电影：' + item_index + ' | ' + item_name +' | ' + item_img +' | ' + item_score +' | ' + item_author +' | ' + item_intr )
        print('爬取电影：' + ' | ' + item_name +' | ' + item_img +' | ' + item_score +' | ' + item_author +' | ' + item_intr )

        global n

        sheet.write(n, 0, item_name)
        sheet.write(n, 1, item_img)
        # sheet.write(n, 2, item_index)
        sheet.write(n, 3, item_score)
        sheet.write(n, 4, item_author)
        sheet.write(n, 5, item_intr)

        n = n + 1


def main(page):
    url = 'https://movie.douban.com/top250?start=' + str(page * 25) + '&filter='
    html = request_douban(url)
    soup = BeautifulSoup(html, 'lxml')
    save_to_excel(soup)


if __name__ == '__main__':

    for i in range(0, 10):
        main(i)

book.save(u'豆瓣最受欢迎的250部电影.xlsx')

爬取电影： | 肖申克的救赎 | https://img3.doubanio.com/view/photo/s_ratio_poster/public/p480747492.jpg | 9.7 | 
                            导演: 弗兰克·德拉邦特 Frank Darabont   主演: 蒂姆·罗宾斯 Tim Robbins /...
                            1994 / 美国 / 犯罪 剧情
                         | NOT AVAILABLE
爬取电影： | 霸王别姬 | https://img1.doubanio.com/view/photo/s_ratio_poster/public/p2561716440.jpg | 9.6 | 
                            导演: 陈凯歌 Kaige Chen   主演: 张国荣 Leslie Cheung / 张丰毅 Fengyi Zha...
                            1993 / 中国大陆 中国香港 / 剧情 爱情 同性
                         | NOT AVAILABLE
爬取电影： | 泰坦尼克号 | https://img9.doubanio.com/view/photo/s_ratio_poster/public/p457760035.jpg | 9.5 | 
                            导演: 詹姆斯·卡梅隆 James Cameron   主演: 莱昂纳多·迪卡普里奥 Leonardo...
                            1997 / 美国 墨西哥 / 剧情 爱情 灾难
                         | NOT AVAILABLE
爬取电影： | 阿甘正传 | https://img3.doubanio.com/view/photo/s_ratio_poster/public/p2372307693.jpg | 9.5 | 
                            导演: 罗伯特·泽米吉斯 Robert Zemeckis   主演: 汤姆

In [19]:
import requests
from bs4 import BeautifulSoup
import csv  # Import the csv module


def request_douban(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                      'Chrome/88.0.4324.146 Safari/537.36',
    }

    try:
        response = requests.get(url=url, headers=headers)
        if response.status_code == 200:
            return response.text
    except requests.RequestException:
        return None


# Open the CSV file in write mode
with open('douban_movies.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)

    # Write the header row
    writer.writerow(['名称', '图片', '评分', '作者', '简介'])

    def save_to_csv(soup):  # Changed function name to reflect CSV saving
        list = soup.find(class_='grid_view').find_all('li')

        for item in list:
            item_name = item.find(class_='title').string
            item_img = item.find('a').find('img').get('src')
            item_score = item.find(class_='rating_num').string
            item_author = item.find('p').text
            if item.find(class_='inq') is not None:
                item_intr = item.find(class_='inq').string
            else:
                item_intr = 'NOT AVAILABLE'

            print('爬取电影：' + ' | ' + item_name + ' | ' + item_img + ' | ' + item_score + ' | ' + item_author + ' | ' + item_intr)

            # Write data to the CSV file
            writer.writerow([item_name, item_img, item_score, item_author, item_intr])

    def main(page):
        url = 'https://movie.douban.com/top250?start=' + str(page * 25) + '&filter='
        html = request_douban(url)
        soup = BeautifulSoup(html, 'lxml')
        save_to_csv(soup)  # Call the save_to_csv function

    if __name__ == '__main__':
        for i in range(0, 10):
            main(i)

爬取电影： | 肖申克的救赎 | https://img3.doubanio.com/view/photo/s_ratio_poster/public/p480747492.jpg | 9.7 | 
                            导演: 弗兰克·德拉邦特 Frank Darabont   主演: 蒂姆·罗宾斯 Tim Robbins /...
                            1994 / 美国 / 犯罪 剧情
                         | NOT AVAILABLE
爬取电影： | 霸王别姬 | https://img1.doubanio.com/view/photo/s_ratio_poster/public/p2561716440.jpg | 9.6 | 
                            导演: 陈凯歌 Kaige Chen   主演: 张国荣 Leslie Cheung / 张丰毅 Fengyi Zha...
                            1993 / 中国大陆 中国香港 / 剧情 爱情 同性
                         | NOT AVAILABLE
爬取电影： | 泰坦尼克号 | https://img9.doubanio.com/view/photo/s_ratio_poster/public/p457760035.jpg | 9.5 | 
                            导演: 詹姆斯·卡梅隆 James Cameron   主演: 莱昂纳多·迪卡普里奥 Leonardo...
                            1997 / 美国 墨西哥 / 剧情 爱情 灾难
                         | NOT AVAILABLE
爬取电影： | 阿甘正传 | https://img3.doubanio.com/view/photo/s_ratio_poster/public/p2372307693.jpg | 9.5 | 
                            导演: 罗伯特·泽米吉斯 Robert Zemeckis   主演: 汤姆